# Pytest

## Definition
Das Pytest Framework wird dazu verwendet Unit-Tests in Python umzusetzen.<br>
In diesem Notebook wird für die Demonstration die ipytest Bibliothek verwendet. (https://github.com/chmp/ipytest)<br>
Diese ermöglicht es pytests über magics Befehle auszuführen.

## Setup
Um die ipytest Bibliothek zu installieren muss folgender Befehl in der Singularity Shell ausgeführt werden:<br>
`pip install ipytest`

In [ ]:
import ipytest
import pytest
import sys
ipytest.autoconfig()

Durch `ipytest.autoconfig()` werden verschiedene Konfigurationen durchgeführt, die für das Verwenden von dem Framework notwendig sind. Natürlich können diese auch manuell gesetzt werden, oder es können Werte angepasst werden.

## Definieren von Tests
Pytest erkennt Funktionen, welche mit dem Keyword "test" beginnen als Tests.<br>
Über den Befehl `assert` können Ergebnisse von Funktionen mit den erwarteten Ergebnissen verglichen werden. Stimmt die definierte Bedingung, so gilt der Test als Bestanden.<br>
Mit dem Ausführen der Zelle wird dank der ipytest Bibliothek auch der Test ausgeführt und das Ergebnis wird unterhalb der Zelle angezeigt.

In [ ]:
%%ipytest
def increment(x):
    return x + 1

def test_increment():
    assert increment(3) == 4

Funktionen, welche nicht mit "test" beginnen werden auch nicht als solche erkannt.

In [ ]:
%%ipytest
def increment(x):
    return x + 1

def increment_test():
    assert increment(3) == 4

## Definieren von mehreren Tests
Analog zum vorherigen Beispiel können beliebig viele Tests in einer Zelle/Klasse definiert werden. Mit dem Ausführen der Zelle werden alle Tests nacheinander abgearbeitet und als gemeinsames Ergebnis am Ende dargestellt.

In [ ]:
%%ipytest
def decrease(x):
    return x - 1

def test_decrease_1():
    assert decrease(3) == 2

def test_decrease_2():
    assert decrease(3) < 3

Mehrere `assert` Befehle in einer Testfunktion sind möglich. Diese zählen dann jedoch zusammen als ein einzelner Test. Das heißt, wenn ein `assert` fehlschlägt, dann betrifft das den gesamten Test.

In [ ]:
%%ipytest
def multiply(x, y):
    return x * y

def test_multiply():
    assert multiply(2, 4) == 8
    assert multiply(2, 4) > (2 + 4)
    assert multiply(2, 4) < 9

## Prüfen auf Exceptions
Es kann unter Umständen sinnvoll sein zu überprüfen ob gewisse Exceptions an gewollten Stellen geworfen werden.<br>
Dafür stellt Pytest den `with` Befehl zur Verfügung. Mit `with pytest.raises(ExceptionXYZ...)` kann überprüft werden ob diese Spezielle Exception geworfen wird.

In [ ]:
%%ipytest
def divide(x, y):
    return x / y

def test_divide():
    with pytest.raises(ZeroDivisionError):
        divide(1, 0)

Falls mehrere Exceptions über bestimmte Keywords zusammengefasst werden können, kann über einen Regex-Ausdruck geprüft werden, ob dieser in der Exception auftritt.

In [ ]:
%%ipytest
def func(x):
    if x == 1:
        raise ValueError("Exception 123 1")
    elif x == 2:
        raise ValueError("Exception 123 2")
        
def test_func1():
    with pytest.raises(ValueError, match=r".* 123 .*"):
        func(1)
        
def test_func2():
    with pytest.raises(ValueError, match=r".* 123 .*"):
        func(2)

## Pytest Marker
Pytest bietet verschiedene Möglichkeiten an Tests vorweg zu markieren, um ihnen ein entsprechendes Verhalten zuzuweisen.<br>
`@pytest.mark.skip` überspringt den Test. Dieser wird nicht ausgeführt und somit ist auch das potentielle Ergebnis irrelevant.

In [ ]:
%%ipytest

@pytest.mark.skip(reason='this is broken')
def test_skip():
    pass

Über `@pytest.mark.skipif(...)` kann eine Bedingung angegeben werden unter welcher der Test übersprungen werden soll.

In [ ]:
%%ipytest

@pytest.mark.skipif(sys.platform == 'linux', reason='Windows behaviour')
def test_skip_conditionally():
    pass

`@pytest.mark.xfail` gibt an, dass erwartet wird, dass der Test fehlschlägt. (Expected Failure)

In [ ]:
%%ipytest

@pytest.mark.xfail
def test_expected_failure():
    assert False

Sollte ein Test, welcher als `xfail` markiert ist, dennoch auf mysteriöse weise erfolgreich sein, wird dieser als `xpass` gewertet. (Unexpectedly Passed)

In [ ]:
%%ipytest

@pytest.mark.xfail
def test_expected_failure_but_passed_mysteriously():
    pass

## Tracebacks
Über folgende Parameteter kann der Traceback im Output konfiguriert werden:
```
--tb=long
--tb=short
--tb=line
--tb=native
--tb=no
``` 

In [ ]:
%%ipytest --tb=long
def func(x):
    return x + x

def test_func1():
    assert func(1) == 1
    
def test_func2():
    assert func(2) == 2
    
def test_func3():
    assert func(3) == 3
    
def test_func4():
    assert func(4) == 4

## Maximale Fails
Über `--maxfail=x` kann angegeben werden wie viele Tests fehlschlagen dürfen bevor die Tests abgebrochen werden.

In [ ]:
%%ipytest --maxfail=2
def func(x):
    return x + x

def test_func1():
    assert func(1) == 1
    
def test_func2():
    assert func(2) == 2
    
def test_func3():
    assert func(3) == 3
    
def test_func4():
    assert func(4) == 4

## Mögliche Testergebnisse
Hier werden alle möglichen Testergebnisse noch einmal zusammengefasst dargestellt.

In [ ]:
%%ipytest -ra

@pytest.fixture
def error_fixture():
    assert 0


def test_ok():
    print("ok")


def test_fail():
    assert 0


def test_error(error_fixture):
    pass


def test_skip():
    pytest.skip("skipping this test")


def test_xfail():
    pytest.xfail("xfailing this test")


@pytest.mark.xfail(reason="always xfail")
def test_xpass():
    pass